In [6]:
pip install astropy pymongo pyspark matplotlib pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pymongo
import pandas as pd
from pyspark.sql import SparkSession
from astropy.io import fits
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import numpy as np

# Conexión a MongoDB
def setup_database():
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client["wmap_database"]
    return db

# Poblar la base de datos desde los archivos FITS
def populate_database(db, fits_files):
    for file in fits_files:
        with fits.open(file) as hdul:
            data = hdul[1].data
            collection = db["wmap_data"]
            for row in data:
                document = {key: row[key] for key in row.names}
                collection.insert_one(document)
    print("Base de datos poblada exitosamente.")

# Consultas avanzadas en MongoDB
def advanced_queries(db):
    collection = db["wmap_data"]
    # Ejemplo 1: Promedio de temperaturas
    avg_temp = collection.aggregate([
        {"$group": {"_id": None, "average_temp": {"$avg": "$TEMPERATURE"}}}
    ])
    print("Promedio de temperatura:", list(avg_temp))
    
    # Ejemplo 2: Regiones con anomalías térmicas
    anomalies = collection.find({"TEMPERATURE": {"$gt": 50}})
    print("Anomalías:", list(anomalies))
    
    # Ejemplo 3: Agrupación por frecuencia
    freq_groups = collection.aggregate([
        {"$group": {"_id": "$FREQUENCY", "count": {"$sum": 1}}}
    ])
    print("Agrupación por frecuencia:", list(freq_groups))

# Procesamiento distribuido con Spark
def spark_processing(fits_files):
    spark = SparkSession.builder \
        .appName("WMAP Analysis") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    
    # Leer los archivos FITS como DataFrame
    data_frames = []
    for file in fits_files:
        with fits.open(file) as hdul:
            data = pd.DataFrame(hdul[1].data)
            data_frames.append(data)
    full_data = pd.concat(data_frames)
    
    # Convertir a Spark DataFrame
    df = spark.createDataFrame(full_data)
    df.createOrReplaceTempView("wmap_table")
    
    # Consulta ejemplo: Histograma de temperaturas
    result = spark.sql("SELECT TEMPERATURE, COUNT(*) AS count FROM wmap_table GROUP BY TEMPERATURE")
    result.show()

# Análisis con Astropy
def astropy_analysis(fits_file):
    with fits.open(fits_file) as hdul:
        data = hdul[1].data
        coords = SkyCoord(data['RA'], data['DEC'], unit="deg")
        # Convertir a coordenadas galácticas
        galactic_coords = coords.galactic
        print("Coordenadas galácticas:", galactic_coords)
    
    # Graficar ejemplo
    plt.hist(data['TEMPERATURE'], bins=50)
    plt.title("Distribución de Temperaturas")
    plt.xlabel("Temperatura (K)")
    plt.ylabel("Frecuencia")
    plt.show()

# Main function
if __name__ == "__main__":
    # Conexión y setup de base de datos
    db = setup_database()
    
    # Archivos FITS de ejemplo (Reemplaza con tu directorio)
    fits_files = ["wmap_mollweide_band_imap_r9_9yr_K_v5.fits"]
    
    # Poblar la base de datos
    populate_database(db, fits_files)
    
    # Realizar consultas avanzadas
    advanced_queries(db)
    
    # Procesamiento distribuido
    spark_processing(fits_files)
    
    # Análisis con Astropy
    astropy_analysis(fits_files[0])
